In [1]:
import pandas as pd
from datasets import load_dataset

/home/tstad/.local/share/hatch/env/virtual/deepset-cloud-custom-nodes/1TRZ-Wjw/deepset-cloud-custom-nodes/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Investigating the source data

In [5]:
load_dataset("hotpot_qa", name="distractor")

Generating validation split: 100%|██████████| 7405/7405 [00:01<00:00, 5214.02 examples/s]


DatasetDict({
    train: Dataset({
        features: ['id', 'question', 'answer', 'type', 'level', 'supporting_facts', 'context'],
        num_rows: 90447
    })
    validation: Dataset({
        features: ['id', 'question', 'answer', 'type', 'level', 'supporting_facts', 'context'],
        num_rows: 7405
    })
})

In [6]:
ds = load_dataset("hotpot_qa", name="distractor")

In [14]:
ds["validation"][0]

{'id': '5a8b57f25542995d1e6f1371',
 'question': 'Were Scott Derrickson and Ed Wood of the same nationality?',
 'answer': 'yes',
 'type': 'comparison',
 'level': 'hard',
 'supporting_facts': {'title': ['Scott Derrickson', 'Ed Wood'],
  'sent_id': [0, 0]},
 'context': {'title': ['Ed Wood (film)',
   'Scott Derrickson',
   'Woodson, Arkansas',
   'Tyler Bates',
   'Ed Wood',
   'Deliver Us from Evil (2014 film)',
   'Adam Collis',
   'Sinister (film)',
   'Conrad Brooks',
   'Doctor Strange (2016 film)'],
  'sentences': [['Ed Wood is a 1994 American biographical period comedy-drama film directed and produced by Tim Burton, and starring Johnny Depp as cult filmmaker Ed Wood.',
    " The film concerns the period in Wood's life when he made his best-known films as well as his relationship with actor Bela Lugosi, played by Martin Landau.",
    ' Sarah Jessica Parker, Patricia Arquette, Jeffrey Jones, Lisa Marie, and Bill Murray are among the supporting cast.'],
   ['Scott Derrickson (born Jul

In [8]:
len(ds["train"])

90447

In [9]:
ds_full = load_dataset("hotpot_qa", name="fullwiki")

Generating test split: 100%|██████████| 7405/7405 [00:01<00:00, 5896.20 examples/s]


In [16]:
ds_full["validation"][0]

{'id': '5a8b57f25542995d1e6f1371',
 'question': 'Were Scott Derrickson and Ed Wood of the same nationality?',
 'answer': 'yes',
 'type': 'comparison',
 'level': 'hard',
 'supporting_facts': {'title': ['Scott Derrickson', 'Ed Wood'],
  'sent_id': [0, 0]},
 'context': {'title': ['Adam Collis',
   'Ed Wood (film)',
   'Tyler Bates',
   'Doctor Strange (2016 film)',
   'Hellraiser: Inferno',
   'Sinister (film)',
   'Deliver Us from Evil (2014 film)',
   'Woodson, Arkansas',
   'Conrad Brooks',
   'The Exorcism of Emily Rose'],
  'sentences': [['Adam Collis is an American filmmaker and actor.',
    ' He attended the Duke University from 1986 to 1990 and the University of California, Los Angeles from 2007 to 2010.',
    ' He also studied cinema at the University of Southern California from 1991 to 1997.',
    ' Collis first work was the assistant director for the Scott Derrickson\'s short "Love in the Ruins" (1995).',
    ' In 1998, he played "Crankshaft" in Eric Koyanagi\'s "Hundred Percen

In [11]:
len(ds_full["train"])

90447

In [18]:
len(ds["validation"]) == len(ds_full["validation"])

True

In [21]:
for disc, full in zip(ds["train"], ds_full["train"]):
    if disc != full:
        print(disc)
        print(full)
        break

In [22]:
for disc, full in zip(ds["validation"], ds_full["validation"]):
    if disc != full:
        print(disc)
        print(full)
        break

{'id': '5a8b57f25542995d1e6f1371', 'question': 'Were Scott Derrickson and Ed Wood of the same nationality?', 'answer': 'yes', 'type': 'comparison', 'level': 'hard', 'supporting_facts': {'title': ['Scott Derrickson', 'Ed Wood'], 'sent_id': [0, 0]}, 'context': {'title': ['Ed Wood (film)', 'Scott Derrickson', 'Woodson, Arkansas', 'Tyler Bates', 'Ed Wood', 'Deliver Us from Evil (2014 film)', 'Adam Collis', 'Sinister (film)', 'Conrad Brooks', 'Doctor Strange (2016 film)'], 'sentences': [['Ed Wood is a 1994 American biographical period comedy-drama film directed and produced by Tim Burton, and starring Johnny Depp as cult filmmaker Ed Wood.', " The film concerns the period in Wood's life when he made his best-known films as well as his relationship with actor Bela Lugosi, played by Martin Landau.", ' Sarah Jessica Parker, Patricia Arquette, Jeffrey Jones, Lisa Marie, and Bill Murray are among the supporting cast.'], ['Scott Derrickson (born July 16, 1966) is an American director, screenwrite

distractor is what we want to use as there we are guaranteed to have the 2 gold passages included

In [37]:
easy_ones = [sample for sample in ds["train"] if sample["level"] == "easy"]
len(easy_ones)

17972

In [41]:
easy_ones[4]

{'id': '5ab698885542995eadef002a',
 'question': 'American politician Joe Heck ran unsuccessfully against Democrat Catherine Cortez Masto, a woman who previously served as the 32nd Attorney General of where?',
 'answer': 'Nevada',
 'type': 'bridge',
 'level': 'easy',
 'supporting_facts': {'title': ['Joe Heck',
   'Joe Heck',
   'Catherine Cortez Masto'],
  'sent_id': [0, 2, 2]},
 'context': {'title': ['Kamala Harris',
   'Maryland Attorney General election, 2014',
   'Corey Palumbo',
   'Catherine Cortez Masto',
   'Joe Heck',
   'List of United States Senators from Nevada',
   'Dennis Heck',
   'Samuel D. Jackson',
   'George Henry Williams',
   'Jeff Harris (politician)'],
  'sentences': [['Kamala Devi Harris ( , ; born October 20, 1964) is an American attorney and politician.',
    ' A member of the Democratic Party, she currently serves as the junior senator from California.',
    ' She previously served as the 32nd Attorney General of California.'],
   ['The Maryland Attorney Gener

# Creating the dataset

In [54]:
ds = load_dataset("hotpot_qa", name="distractor")

In [55]:
example = ds["validation"][0]
example

{'id': '5a8b57f25542995d1e6f1371',
 'question': 'Were Scott Derrickson and Ed Wood of the same nationality?',
 'answer': 'yes',
 'type': 'comparison',
 'level': 'hard',
 'supporting_facts': {'title': ['Scott Derrickson', 'Ed Wood'],
  'sent_id': [0, 0]},
 'context': {'title': ['Ed Wood (film)',
   'Scott Derrickson',
   'Woodson, Arkansas',
   'Tyler Bates',
   'Ed Wood',
   'Deliver Us from Evil (2014 film)',
   'Adam Collis',
   'Sinister (film)',
   'Conrad Brooks',
   'Doctor Strange (2016 film)'],
  'sentences': [['Ed Wood is a 1994 American biographical period comedy-drama film directed and produced by Tim Burton, and starring Johnny Depp as cult filmmaker Ed Wood.',
    " The film concerns the period in Wood's life when he made his best-known films as well as his relationship with actor Bela Lugosi, played by Martin Landau.",
    ' Sarah Jessica Parker, Patricia Arquette, Jeffrey Jones, Lisa Marie, and Bill Murray are among the supporting cast.'],
   ['Scott Derrickson (born Jul

In [56]:
gold_titles = example["supporting_facts"]["title"]
gold_context_idxs = [example["context"]["title"].index(gold_title) for gold_title in gold_titles]
gold_contexts = ["".join(example["context"]["sentences"][idx]) for idx in gold_context_idxs]
distractor_context_ids = [idx for idx in range(len(example["context"]["title"])) if idx not in gold_context_idxs]
distractor_contexts = ["".join(example["context"]["sentences"][idx]) for idx in distractor_context_ids]
distractor_contexts

["Ed Wood is a 1994 American biographical period comedy-drama film directed and produced by Tim Burton, and starring Johnny Depp as cult filmmaker Ed Wood. The film concerns the period in Wood's life when he made his best-known films as well as his relationship with actor Bela Lugosi, played by Martin Landau. Sarah Jessica Parker, Patricia Arquette, Jeffrey Jones, Lisa Marie, and Bill Murray are among the supporting cast.",
 'Woodson is a census-designated place (CDP) in Pulaski County, Arkansas, in the United States. Its population was 403 at the 2010 census. It is part of the Little Rock–North Little Rock–Conway Metropolitan Statistical Area. Woodson and its accompanying Woodson Lake and Wood Hollow are the namesake for Ed Wood Sr., a prominent plantation owner, trader, and businessman at the turn of the 20th century. Woodson is adjacent to the Wood Plantation, the largest of the plantations own by Ed Wood Sr.',
 'Tyler Bates (born June 5, 1965) is an American musician, music produce

In [57]:
context_relevance_ds = []
for example in ds["validation"]:
    gold_titles = example["supporting_facts"]["title"]
    gold_context_idxs = [example["context"]["title"].index(gold_title) for gold_title in gold_titles]
    gold_contexts = ["".join(example["context"]["sentences"][idx]) for idx in gold_context_idxs]
    distractor_context_ids = [idx for idx in range(len(example["context"]["title"])) if idx not in gold_context_idxs]
    distractor_contexts = ["".join(example["context"]["sentences"][idx]) for idx in distractor_context_ids]
    for gold_context in gold_contexts:
        context_relevance_ds.append(
            {
                "id": example["id"],
                "source": "hotpot_qa/distractor/validation",
                "question": example["question"],
                "answer": example["answer"],
                "context": gold_context,
                "relevance": "partial",
                "relevance_value": 0.5,
                "type": "gold_context_part",
            }
        )
    context_relevance_ds.append(
        {
            "id": example["id"],
            "source": "hotpot_qa/distractor/validation",
            "question": example["question"],
            "answer": example["answer"],
            "context": "\n".join(gold_contexts),
            "relevance": "full",
            "relevance_value": 1.0,
            "type": "gold_context_full",
        }
    )
    for distractor_context in distractor_contexts:
        context_relevance_ds.append(
            {
                "id": example["id"],
                "source": "hotpot_qa/distractor/validation",
                "question": example["question"],
                "answer": example["answer"],
                "context": distractor_context,
                "relevance": "not_relevant",
                "relevance_value": 0.0,
                "type": "distractor_context_part",
            }
        )
    context_relevance_ds.append(
        {
            "id": example["id"],
            "source": "hotpot_qa/distractor/validation",
            "question": example["question"],
            "answer": example["answer"],
            "context": "\n".join(distractor_contexts),
            "relevance": "not_relevant",
            "relevance_value": 0.0,
            "type": "distractor_context_full",
        }
    )

In [58]:
len(context_relevance_ds)

91705

In [59]:
context_relevance_ds[0]

{'id': '5a8b57f25542995d1e6f1371',
 'source': 'hotpot_qa/distractor/validation',
 'question': 'Were Scott Derrickson and Ed Wood of the same nationality?',
 'answer': 'yes',
 'context': 'Scott Derrickson (born July 16, 1966) is an American director, screenwriter and producer. He lives in Los Angeles, California. He is best known for directing horror films such as "Sinister", "The Exorcism of Emily Rose", and "Deliver Us From Evil", as well as the 2016 Marvel Cinematic Universe installment, "Doctor Strange."',
 'relevance': 'partial',
 'relevance_value': 0.5,
 'type': 'gold_context_part'}

# Building the Pipeline

Prompt has been checked with gpt-4o and Claude Sonnet 3.5. It seems to work quite well.

In [22]:
from haystack.components.builders import PromptBuilder

template= """You are evaluating a search system.
You have to decide, if a retrieved paragraph is relevant to the user's question.

A relevant paragraph is a paragraph that helps to answers the question.

Use the following rating scheme:
1.0: covers full answer
0.5: contributes to the answer
0.0: not relevant

Think step by step. First try to answer the question. If you can, you have to rate 1.0. If not decide between 0.5 and 0.0.
Describe why you chose the rating. Do not repeat the rating scheme.
Finally, put out your rating of the relevance as number using the following notation without any markdown or text around it:

"Rating: "

Here is the paragraph:

{{ context }}

Here is the question: {{ question }}

"""

prompt_builder = PromptBuilder(template=template)

In [23]:
import os
from haystack.components.generators import OpenAIGenerator

os.environ["OPENAI_API_KEY"] = "INSERT_OPENAI_API_KEY_HERE"

generator = OpenAIGenerator(model="gpt-4o", generation_kwargs={"temperature": 0.0})

In [24]:
import os
from haystack_integrations.components.generators.amazon_bedrock.generator import AmazonBedrockGenerator

# insert AWS config here
os.environ["AWS_PROFILE"] = ""
os.environ["AWS_DEFAULT_REGION"] = "us-east-1"

generator = AmazonBedrockGenerator(model="anthropic.claude-3-5-sonnet-20240620-v1:0", max_length=1000, temperature=0.0)

In [25]:
from deepset_cloud_custom_nodes import DeepsetAnswerBuilder

reasoning_extractor = DeepsetAnswerBuilder()
rating_extractor = DeepsetAnswerBuilder(pattern=r"Rating: ([0-9.]+)")

In [26]:
from haystack import Pipeline

pipeline = Pipeline()
pipeline.add_component(instance=prompt_builder, name="prompt_builder")
pipeline.add_component(instance=generator, name="generator")
pipeline.add_component(instance=rating_extractor, name="rating_extractor")
pipeline.add_component(instance=reasoning_extractor, name="reasoning_extractor")

In [27]:
pipeline.connect("prompt_builder", "generator")
pipeline.connect("generator.replies", "rating_extractor.replies")
pipeline.connect("generator.replies", "reasoning_extractor.replies")
pipeline.connect("prompt_builder.prompt", "rating_extractor.prompt")
pipeline.connect("prompt_builder.prompt", "reasoning_extractor.prompt")

🚅 Components
  - prompt_builder: PromptBuilder
  - generator: AmazonBedrockGenerator
  - rating_extractor: DeepsetAnswerBuilder
  - reasoning_extractor: DeepsetAnswerBuilder
🛤️ Connections
  - prompt_builder.prompt -> generator.prompt (str)
  - prompt_builder.prompt -> rating_extractor.prompt (str)
  - prompt_builder.prompt -> reasoning_extractor.prompt (str)
  - generator.replies -> rating_extractor.replies (List[str])
  - generator.replies -> reasoning_extractor.replies (List[str])

In [82]:
example = context_relevance_ds[0]
question = example["question"]
context = example["context"]

res = pipeline.run(data={
    "prompt_builder": {"question": question, "context": context}, 
    "rating_extractor": {"query": question},
    "reasoning_extractor": {"query": question},
    }, include_outputs_from=["rating_extractor", "reasoning_extractor", "generator", "prompt_builder"])

In [83]:
res

{'rating_extractor': {'answers': [GeneratedAnswer(data='0.5', query='Were Scott Derrickson and Ed Wood of the same nationality?', documents=[], meta={'prompt': 'You are evaluating a search system.\nYou have to decide, if a retrieved paragraph is relevant to the user\'s question.\n\nA relevant paragraph is a paragraph that helps to answers the question.\n\nUse the following rating scheme:\n1.0: covers full answer\n0.5: contributes to the answer\n0.0: not relevant\n\nThink step by step. First try to answer the question. If you can, you have to rate 1.0. If not decide between 0.5 and 0.0.\nDescribe why you chose the rating. Do not repeat the rating scheme.\nFinally, put out your rating of the relevance as number using the following notation without any markdown or text around it:\n\n"Rating: "\n\nHere is the paragraph:\n\nScott Derrickson (born July 16, 1966) is an American director, screenwriter and producer. He lives in Los Angeles, California. He is best known for directing horror film

In [86]:
ids = list(set([example["id"] for example in context_relevance_ds]))

# sample 50 with seed
import random

random.seed(42)
sample_ids = random.sample(ids, 250)

sampled_context_relevance_ds = [example for example in context_relevance_ds if example["id"] in sample_ids]

In [87]:
len(sampled_context_relevance_ds)

3100

# Run pipeline

In [ ]:
import pandas as pd
from tqdm import tqdm

for example in tqdm(sampled_context_relevance_ds):
    question = example["question"]
    context = example["context"]
    res = pipeline.run(data={
        "prompt_builder": {"question": question, "context": context}, 
        "rating_extractor": {"query": question},
        "reasoning_extractor": {"query": question},
        }, include_outputs_from=["rating_extractor", "reasoning_extractor", "generator", "prompt_builder"])
    example["llm_judge_rating"] = res["rating_extractor"]["answers"][0].data
    example["llm_judge_reasoning"] = res["reasoning_extractor"]["answers"][0].data
    example["llm_judge_prompt"] = res["prompt_builder"]["prompt"]

    df = pd.DataFrame(sampled_context_relevance_ds)
    df.to_csv("../../data/context_relevance/llm_judge_hotpot_qa_distractor_validation_sonnet3.5.csv", index=False)

In [74]:
right, wrong, error, very_wrong = 0, 0, 0, 0
wrongs = []
very_wrongs = []
errors = []
for sample in sampled_context_relevance_ds:
    if sample["llm_judge_rating"] == "":
        error += 1
        errors.append(sample)
    elif float(sample["llm_judge_rating"]) == sample["relevance_value"]:
        right += 1
    else:
        wrong += 1
        wrongs.append(sample)
        if abs(float(sample["llm_judge_rating"]) - sample["relevance_value"]) >= 1.0:
            very_wrong += 1
            very_wrongs.append(sample)

In [75]:
right, wrong, error, very_wrong

(2113, 987, 0, 50)

In [76]:
very_wrongs_avg = sum([float(sample["llm_judge_rating"]) - sample["relevance_value"] for sample in very_wrongs]) / len(very_wrongs)
very_wrongs_avg

0.96

In [77]:
wrongs_avg = sum([float(sample["llm_judge_rating"]) - sample["relevance_value"] for sample in wrongs]) / len(wrongs)
wrongs_avg

0.4746707193515704

In [82]:
to_inspect = wrongs[985]
to_inspect

{'id': '5a7e910a554299306751357f',
 'source': 'hotpot_qa/distractor/validation',
 'question': "Who was the artist that recorded a duet in 1985 for the CBS TV series 'Dallas' with an artist that was born in 1951 and has twenty #1 hits?",
 'answer': 'Gary Morris',
 'context': 'Louis Jordan was an American popular music innovator who recorded from the 1930s until the 1970s. During the 1940s, he was the most popular recording artist of the soon-to-be-called rhythm and blues music. Jordan had eighteen No. 1 hits, which places him as the third most successful singles artist in "Billboard" R&B charts history. His 1946 recording of "Choo Choo Ch\'Boogie" is tied for first place for spending the most weeks (eighteen) at No. 1. Jordan\'s success was not limited to the R&B market — he also had No. 1 hits on the Billboard Pop and Country charts.',
 'relevance': 'not_relevant',
 'relevance_value': 0.0,
 'type': 'distractor_context_part',
 'llm_judge_rating': 0.5,
 'llm_judge_reasoning': "Let's appr

In [83]:
print(to_inspect["context"])

Louis Jordan was an American popular music innovator who recorded from the 1930s until the 1970s. During the 1940s, he was the most popular recording artist of the soon-to-be-called rhythm and blues music. Jordan had eighteen No. 1 hits, which places him as the third most successful singles artist in "Billboard" R&B charts history. His 1946 recording of "Choo Choo Ch'Boogie" is tied for first place for spending the most weeks (eighteen) at No. 1. Jordan's success was not limited to the R&B market — he also had No. 1 hits on the Billboard Pop and Country charts.


In [84]:
print(to_inspect["llm_judge_reasoning"])

Let's approach this step-by-step:

1. First, I'll try to answer the question based on the given paragraph.

2. The question asks about an artist who recorded a duet in 1985 for 'Dallas' with someone born in 1951 who has twenty #1 hits.

3. The paragraph doesn't mention anything about:
   - A duet recorded in 1985
   - The TV series 'Dallas'
   - An artist born in 1951
   - Anyone with twenty #1 hits

4. The paragraph is about Louis Jordan, who was active from the 1930s to the 1970s, which doesn't align with the 1985 timeframe in the question.

5. While the paragraph does mention #1 hits (Jordan had eighteen), this doesn't match the twenty #1 hits mentioned in the question.

6. Given these points, I cannot answer the question based on this paragraph.

7. However, the paragraph does provide some information about a successful R&B artist (Louis Jordan) and mentions #1 hits, which is somewhat related to the concept of successful music artists and chart performance.

8. While not directly r